In [14]:
import pandas as pd;
import numpy as np;
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import scipy.stats as stats

plt.rcParams.update({'font.size': 14})
pd.set_option('display.float_format', lambda x: '%.2f' % x)
path = '../../../../playwright/results/core-web-vitals/testrun-8/'


In [15]:
df = pd.read_csv(f'{path}2024-12-16T09:36:50.835Z_results-csr.csv', sep=',')
df_t = pd.read_csv(f'{path}2024-12-16T11:08:37.924Z_results-csr-throttled.csv', sep=',')
df_ht = pd.read_csv(f'{path}2024-12-16T12:37:45.058Z_results-csr-throttled-heavy.csv', sep=',')

features = ['navTime', 'totalTime', 'lcp', 'fcp', 'ttfb', 'tbt', 'tti', 'longestTask', 'longTasks', 'nf:init', 'nf:config','nf:loaded']

dirty_dfs = {
    'unthrottled': df[5:],
    'throttled': df_t[5:],
    'heavy': df_ht[5:],
}

In [16]:
def detect_outliers(_df, _features, contamination=0.1):
    clf = IsolationForest(contamination=contamination, random_state=42)
    outliers = clf.fit_predict(_df[_features])
    return outliers == 1

masks = {}
dfs = {}
target_features = ['navTime', 'totalTime', 'lcp', 'fcp', 'ttfb']

for name, _df in dirty_dfs.items():
    mask = detect_outliers(_df, target_features)
    masks[name] = mask
    dfs[name] = _df[mask].copy()

In [17]:
columns = [ 'ttfb','fcp','nf:init','lcp','tti','nf:loaded','tbt','longestTask']
rows = []

for name, df in dfs.items():
    mean_row = df[columns].mean()
    percentile_row = df[columns].quantile(0.75)
    
    rows.append((f"{name} (mean)", mean_row))
    rows.append((f"{name} (75th)", percentile_row))

result_df = pd.DataFrame([row[1] for row in rows], index=[row[0] for row in rows])
result_df = result_df.mask(result_df < 0, '-')
result_df

,ttfb,fcp,nf:init,lcp,tti,nf:loaded,tbt,longestTask
unthrottled (mean),3.29,65.77,37.18,133.48,65.77,101.28,0.00,-
unthrottled (75th),3.50,68.50,39.10,137.00,68.50,103.90,0.00,-
throttled (mean),340.01,1131.01,1341.10,4543.92,3776.53,3819.91,34.79,84.79
throttled (75th),340.30,1135.90,1345.80,4553.88,3783.17,3825.98,35.00,85.00
heavy (mean),341.92,1209.22,1477.83,4806.32,4251.78,4252.08,209.73,218.84
heavy (75th),342.50,1214.30,1484.30,4863.18,4260.88,4261.10,213.00,221.00
